In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
courses_dataset_path = "/Users/gowthami/Documents/pythonProjects/DigitalMentorUS/data/UMBC_COEIT_courses_dataset.csv"
courses_dataset = pd.read_csv(courses_dataset_path)
courses_dataset

,Student ID,Interests,Career Goals,Matched Courses
0,1,Software Development,Project Manager,Data Structures
1,2,Management,Network Administrator,Chemical Process Principles
2,3,Computing Basics,Business Analyst,Network Technologies
3,4,Networking,Mechanical Engineer,Database Management Systems
4,5,Software Development,Project Manager,Introduction to Business Technology
...,...,...,...,...
1995,1996,Business Technology,Mechanical Engineer,Introduction to Computing
1996,1997,Chemical Processes,Robotics Engineer,Introduction to Business Technology
1997,1998,Digital Systems,Robotics Engineer,Database Management Systems
1998,1999,Networking,System Architect,Data Structures


In [9]:
print(courses_dataset['Career Goals'].nunique())
print(courses_dataset['Interests'].nunique())
print(courses_dataset['Matched Courses'].nunique())

10
10
10


In [11]:
# Combine 'Interests' and 'Career Goals' into one string per student
courses_dataset['profile_text'] = courses_dataset['Interests'] + " " + courses_dataset['Career Goals']

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
profile_vectors = vectorizer.fit_transform(courses_dataset['profile_text'])
course_vectors = vectorizer.fit_transform(courses_dataset['Matched Courses'])

In [13]:
# Calculate cosine similarity between each student profile and course description
similarity_matrix = cosine_similarity(profile_vectors, course_vectors)

# Generate course recommendations based on highest similarity
top_course_indices = similarity_matrix.argsort(axis=1)[:, -1:]  # Top 3 courses for each student
course_recommendations = [[courses_dataset['Matched Courses'].iloc[idx] for idx in reversed(indices)] for indices in top_course_indices]

courses_dataset['Recommended Courses'] = course_recommendations
courses_dataset[['Interests', 'Career Goals', 'Recommended Courses']]

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 24 while Y.shape[1] == 19

In [17]:
def recommend_courses(interests, career_goals, vectorizer, course_df, course_vectors):
    # Combine the student's interests and career goals into a single profile text
    profile_text = interests + " " + career_goals
    
    # Vectorize the new student profile using the already fitted vectorizer
    profile_vector = vectorizer.transform([profile_text])
    
    # Calculate cosine similarity between the student profile and all course descriptions
    similarity_scores = cosine_similarity(profile_vector, course_vectors)
    
    # Get the indices of the top 3 courses
    top_indices = similarity_scores.argsort()[0][-1:]
    
    # Retrieve the course names based on these indices
    recommended_courses = [course_df['Matched Courses'].iloc[index] for index in reversed(top_indices)]
    
    return recommended_courses

In [23]:
# Example student data
new_student_interests = "Computing Basics"
new_student_career_goals = "Software Engineer"

# Get course recommendations
recommendations = recommend_courses(new_student_interests, new_student_career_goals, vectorizer, courses_dataset, course_vectors)

print("Recommended Courses for the new student:", recommendations)

Recommended Courses for the new student: ['Introduction to Computing']
